In [ ]:
#directory of the dataset for classification
base_dir = '/binaryclass/rhinorm'


train_dir = '/binaryclass/rhinorm/train'
validation_dir = '/binaryclass/rhinorm/validation'
test_dir = '/binaryclass/rhinorm/test'


In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.regularizers import l1, l2, l1_l2


# Set the input image size and number of classes
img_size = (224, 224)
num_classes = 2

# Set the batch size and number of epochs
batch_size = 32
epochs = 20


# Create an ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

# below two lines are commented because it is getting done in val_gen and test_gen
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create ImageDataGenerators for the train, validation, and test sets
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

val_gen = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)


# Set regularization factor
regularization_factor = 0.01
# MobileNetV3 Large model
base_model = MobileNetV3Large(input_shape=img_size + (3,), include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
# Add BatchNormalization layer
x = BatchNormalization()(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(regularization_factor))(x)
# Add BatchNormalization layer
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

output = Dense(1, activation='sigmoid', kernel_regularizer=l2(regularization_factor))(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs=output)




# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False



optimizer = Nadam(learning_rate=1e-5)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()



# Train the model
early_stopping = [EarlyStopping(monitor='val_loss', patience=3),
             ModelCheckpoint(filepath='/binaryclass_output/mobilenet_v3large/mobilenet_v3_binaclass_best_model.h5', monitor='val_loss', save_best_only=True)]

#log the training epochs
log_path = "/binaryclass_output/mobilenet_v3large/mobilenet_v3large_binaclass.log"
csv_callback = tf.keras.callbacks.CSVLogger(log_path, separator=',', append=True)


# Train the model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    verbose=1,
    callbacks=[early_stopping, csv_callback]
)


# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_gen)
print('Test accuracy:', test_acc)

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('/binaryclass_output/mobilenet_v3large/mobilenet_v3large_binaclass_accuracy_plot.png', dpi=300, bbox_inches='tight')
plt.show()

# Plot training & validation loss values
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('/binaryclass_output/mobilenet_v3large/mobilenet_v3large_binaclass_loss_plot.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#to get the class labels assigned by the model during training.
class_dict = train_gen.class_indices
print(class_dict)

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#test dir
test_dir = '/rhinorm/test'

# Load the saved model .h5 file
loaded_model = load_model('/binaryclass_output/mobilenet_v3large/mobilenet_v3_binaclass_best_model.h5')

# Create the test generator with the same preprocessing as used during training
test_datagen = ImageDataGenerator(rescale=1./255)

# Prepare the test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,  # Replace with the path to your test directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

# Evaluate the loaded model on the test data
test_loss, test_accuracy = loaded_model.evaluate(test_generator)

# Print the test score (loss and accuracy)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

# Write the test score (loss and accuracy) to a file
with open('/mobilenet_v3large_binaclass_test_results.txt', 'w') as file:
    file.write(f'Test loss: {test_loss}\n')
    file.write(f'Test accuracy: {test_accuracy}\n')


In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# Get true labels and predicted labels from the test set
true_labels = test_generator.classes
predicted_probs = loaded_model.predict(test_generator)
predicted_labels = np.round(predicted_probs).flatten().astype(int)

# Calculate the classification report
report = classification_report(true_labels, predicted_labels, output_dict=True, target_names=['norm', 'rhi'])
report_str = classification_report(true_labels, predicted_labels, target_names=['norm', 'rhi'])

# Print the classification report
print(report_str)

# Save classification report to file
with open('/mobilenet_v3large_binaclass_classification_report.txt', 'w') as f:
     f.write(report_str)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Predict class labels on the test set
y_pred_probs = loaded_model.predict(test_generator)
y_pred = np.round(y_pred_probs).flatten()

# Get the true class labels from the test set
y_true = test_generator.classes

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Print the confusion matrix as an array
print(cm)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Predict class labels on the test set
y_pred_probs = loaded_model.predict(test_generator)
y_pred = np.round(y_pred_probs).flatten()  # Round probabilities to get class labels

# Get the true class labels from the test set
y_true = test_generator.classes

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix using seaborn
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, square=True, xticklabels=['norm', 'rhi'], yticklabels=['norm', 'rhi'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('/mobilenet_v3large_binaclass_cmatrix.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
###################################
##################################
#################################
# BELOW IS FOR TESTING THE RESULTS ON REAL RHINOPHYMA IMAGES
#################################
##################################
###################################

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


test_dir = '/real_rhinorm/test'

# Load the saved model .h5 file
loaded_model = load_model('/mobilenet_v3_binaclass_best_model.h5')

# Prepare the test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,  # Replace with the path to your test directory
    target_size=(224, 224),
    batch_size=8,
    class_mode='binary',
    shuffle=False)

# Evaluate the loaded model on the test data
test_loss, test_accuracy = loaded_model.evaluate(test_generator)

# Print the test score (loss and accuracy)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

# Write the test score (loss and accuracy) to a file
with open('/mobilenet_v3large_binaclass_test_results_REAL.txt', 'w') as file:
    file.write(f'Test loss: {test_loss}\n')
    file.write(f'Test accuracy: {test_accuracy}\n')

In [ ]:
#CLASSIFICATION REPORT on real

import numpy as np
from sklearn.metrics import classification_report

# Get true labels and predicted labels from the test set
true_labels = test_generator.classes
predicted_probs = loaded_model.predict(test_generator)
predicted_labels = np.round(predicted_probs).flatten().astype(int)

# Calculate the classification report
report = classification_report(true_labels, predicted_labels, output_dict=True, target_names=['norm', 'rhi'])
report_str = classification_report(true_labels, predicted_labels, target_names=['norm', 'rhi'])

# Print the classification report
print(report_str)

# Save classification report to file
with open('/mobilenet_v3large_binaclass_classification_report_REAL.txt', 'w') as f:
    f.write(report_str)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Predict class labels on the test set
y_pred_probs = loaded_model.predict(test_generator)
y_pred = np.round(y_pred_probs).flatten()  # Round probabilities to get class labels

# Get the true class labels from the test set
y_true = test_generator.classes

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Print the confusion matrix as an array
print(cm)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Predict class labels on the test set
y_pred_probs = loaded_model.predict(test_generator)
y_pred = np.round(y_pred_probs).flatten()  # Round probabilities to get class labels

# Get the true class labels from the test set
y_true = test_generator.classes

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix using seaborn
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, square=True, xticklabels=['norm', 'rhi'], yticklabels=['norm', 'rhi'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('/mobilenet_v3large_binaclass_cmatrix_REAL.png', dpi=300, bbox_inches='tight')
plt.show()